In [2]:
import datetime
import math
import os
import sys
import ee
from Py6S import *
import ee
import geemap
import ee
import folium
from IPython.display import display
import numpy as np
import geemap
import ee
from IPython.display import display

In [3]:
ee.Authenticate()
ee.Initialize()

Enter verification code: 4/1AfJohXlPce3SRavBiuXTzcvJgAesQXgQOGdcaWa6wioKB_FJFp7-4bnaU7k

Successfully saved authorization token.


In [1]:
import ee
import numpy as np
import matplotlib.pyplot as plt
from geemap import ee_export_image
import time
import ee
import geemap
import os

# Initialize Earth Engine
ee.Initialize()

# Define the region of interest (ROI) as a geometry
table = ee.FeatureCollection("projects/webapp-385310/assets/studyarea_dl").geometry()


# Load the Landsat image collection and filter it
imageCollection = ee.ImageCollection("LANDSAT/LC08/C02/T1") \
    .filterDate('2022-01-01', '2022-02-02') \
    .filterBounds(table)

# Calculate the mean of the filtered images and select the NIR band
meanImage = imageCollection.mean().select('B5')

# Clip the image to the ROI
clippedImage = meanImage#.clip(table)

# Convert the image to 16-bit unsigned integer
grayscale = clippedImage.toUint16()

# Calculate GLCM texture properties
texture = grayscale.glcmTexture()

# Select the contrast, correlation, and dissimilarity bands
contrast = texture.select('B5_contrast')
correlation = texture.select('B5_corr')
dissimilarity = texture.select('B5_diss')

# Define the output folder
output_folder = 'GEE_Exports'


# Define output filenames for contrast, correlation, and dissimilarity
contrast_filename = f'Landsat_contrast.tif'
correlation_filename = f'Landsat_correlation.tif'
dissimilarity_filename = f'Landsat_dissimilarity.tif'

# Export options
export_options = {
    'scale': 30,
    'region': table,
    'crs': 'EPSG:32644',  # Change to your desired CRS
    'maxPixels': 1e13  # Change to an appropriate value
}

# Export contrast, correlation, and dissimilarity to Google Drive with export options
export_contrast_task = ee.batch.Export.image.toDrive(
    image=contrast,
    description='Contrast Export',
    folder=output_folder,
    fileNamePrefix=contrast_filename,
    **export_options  # Include export options
)
export_correlation_task = ee.batch.Export.image.toDrive(
    image=correlation,
    description='Correlation Export',
    folder=output_folder,
    fileNamePrefix=correlation_filename,
    **export_options  # Include export options
)
export_dissimilarity_task = ee.batch.Export.image.toDrive(
    image=dissimilarity,
    description='Dissimilarity Export',
    folder=output_folder,
    fileNamePrefix=dissimilarity_filename,
    **export_options  # Include export options
)

# Start export tasks
export_contrast_task.start()
export_correlation_task.start()
export_dissimilarity_task.start()

print('Exporting GLCM images to Google Drive...')

# Wait for export tasks to complete
while export_contrast_task.active() or export_correlation_task.active() or export_dissimilarity_task.active():
    print('Exporting...')  # You can replace this with a more informative message
    time.sleep(30)  # Wait for 30 seconds before checking again

print('Export completed for GLCM images')

Exporting GLCM images to Google Drive...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Export completed for GLCM images


In [5]:
cloudCover=20

In [7]:
import ee
import os
import time

# Initialize Earth Engine
ee.Initialize()

# Define the region of interest (ROI) as a geometry
table = ee.FeatureCollection("projects/webapp-385310/assets/studyarea_dl").geometry()

# Define the NIR band name for different Landsat sensors
def get_nir_band(landsat_sensor):
    if landsat_sensor in ['LT05', 'LE07']:
        return 'B4'
    elif landsat_sensor in ['LC08', 'LC09']:
        return 'B5'
    else:
        raise ValueError(f"Unsupported Landsat sensor: {landsat_sensor}")

# Loop through the years from 1984 to 2023
for year in range(2004, 2015):
    images_found = False  # Flag to check if images are found for the current year

    # Get the Landsat sensor based on the year
    landsat_sensor = None
    if year < 2000:
        landsat_sensor = 'LT05'
    elif 1999 <= year < 2014:
        landsat_sensor = 'LE07'
    elif 2014 <= year < 2022:
        landsat_sensor = 'LC08'
    else:
        landsat_sensor = 'LC09'

    # Initialize GLCM variables for each texture property
    glcm_mean_contrast = None
    glcm_mean_correlation = None
    glcm_mean_dissimilarity = None
    glcm_mean_energy = None
    glcm_mean_entropy = None
    glcm_count = 0

    # Initialize a flag to check if any NaN values are found
    nan_found = False

    # Iterate through months from January to December
    for month in range(1, 13):
        # Check if NaN values are found in the images
        if nan_found:
            print(f"NaN value found in year {year}, month {month}. Adding the next month and taking the mean.")
        else:
            # Set the start and end dates for the current month
            start_date = f'{year}-01-01'
            end_date = f'{year}-12-28'

            # Filter Landsat collection for the specified year and month
            landsat_collection = ee.ImageCollection(f"LANDSAT/{landsat_sensor}/C02/T1_TOA")

            
            images_1 = landsat_collection.filter(ee.Filter.lt('CLOUD_COVER', cloudCover)) \
                     .filter(ee.Filter.date(start_date, end_date)) \
                     .filterBounds(table)\
                     .filter(ee.Filter.eq('WRS_ROW',41))\
                     .filter(ee.Filter.eq('WRS_ROW',41))\
                     .filter(ee.Filter.eq('WRS_PATH',136))\
                     .mean()
            # Get the date of the first image in images_1
            #first_image_date = ee.Image(images_1).date()
            

            images_2 = landsat_collection.filter(ee.Filter.lt('CLOUD_COVER', cloudCover)) \
                     .filter(ee.Filter.date(start_date, end_date)) \
                     .filterBounds(table)\
                     .filter(ee.Filter.eq('WRS_ROW',42))\
                     .filter(ee.Filter.eq('WRS_ROW',42))\
                     .filter(ee.Filter.eq('WRS_PATH',136))\
                     .mean()
                
            #images = images_1.merge(images_2)
            images = ee.ImageCollection([images_1, images_2])
                
            

            # Check if images are found for the current month
            if images.size().getInfo() > 0:
                images_found = True

                # Take the mean of the images for the specified period
                image = images.mosaic()


                # Select NIR band for GLCM
                nir_band = get_nir_band(landsat_sensor)
                nir_image = image.select(nir_band)

                # Convert NIR band to 32-bit integer
                nir_image_int = nir_image.multiply(10000).toInt32()

                # Calculate GLCM texture properties
                texture = nir_image_int.glcmTexture()
                
                print(texture.getInfo())

                # Select GLCM bands
                contrast = texture.select(get_nir_band(landsat_sensor)+'_contrast')
                correlation = texture.select(get_nir_band(landsat_sensor)+'_corr')
                dissimilarity = texture.select(get_nir_band(landsat_sensor)+'_diss')
                energy = texture.select(get_nir_band(landsat_sensor)+'_energy')
                entropy = texture.select(get_nir_band(landsat_sensor)+'_entropy')

                # Update GLCM means for each texture property
                if glcm_mean_contrast is None:
                    glcm_mean_contrast = contrast
                    glcm_mean_correlation = correlation
                    glcm_mean_dissimilarity = dissimilarity
                    glcm_mean_energy = energy
                    glcm_mean_entropy = entropy
                else:
                    glcm_mean_contrast = glcm_mean_contrast.add(contrast)
                    glcm_mean_correlation = glcm_mean_correlation.add(correlation)
                    glcm_mean_dissimilarity = glcm_mean_dissimilarity.add(dissimilarity)
                    glcm_mean_energy = glcm_mean_energy.add(energy)
                    glcm_mean_entropy = glcm_mean_entropy.add(entropy)
                
                glcm_count += 1

                # Check for NaN values in the contrast image
                if contrast.reduceRegion(reducer=ee.Reducer.anyNonZero(), geometry=table, scale=30).getInfo():
                    nan_found = True
                    print(f"NaN value found in year {year}, month {month}. Adding the next month and taking the mean.")
                    break  # Exit the loop for the current year and month

    # Calculate the mean of GLCM texture properties across all valid months
    if glcm_mean_contrast is not None and glcm_count > 0:
        glcm_mean_contrast = glcm_mean_contrast.divide(glcm_count)
        glcm_mean_correlation = glcm_mean_correlation.divide(glcm_count)
        glcm_mean_dissimilarity = glcm_mean_dissimilarity.divide(glcm_count)
        glcm_mean_energy = glcm_mean_energy.divide(glcm_count)
        glcm_mean_entropy = glcm_mean_entropy.divide(glcm_count)

        # Clip the GLCM images to the ROI
        glcm_mean_contrast = glcm_mean_contrast.clip(table)
        glcm_mean_correlation = glcm_mean_correlation.clip(table)
        glcm_mean_dissimilarity = glcm_mean_dissimilarity.clip(table)
        glcm_mean_energy = glcm_mean_energy.clip(table)
        glcm_mean_entropy = glcm_mean_entropy.clip(table)

        # Define the output folder
        output_folder = 'GEEexport'

        # Define output filenames for GLCM texture properties
        glcm_contrast_filename = f'GLCM_contrast_{year}.tif'
        glcm_correlation_filename = f'GLCM_correlation_{year}.tif'
        glcm_dissimilarity_filename = f'GLCM_dissimilarity_{year}.tif'
        glcm_energy_filename = f'GLCM_energy_{year}.tif'
        glcm_entropy_filename = f'GLCM_entropy_{year}.tif'

        # Export options
        export_options = {
            'scale': 30,
            'region': table,
            'crs': 'EPSG:32644',  # Change to your desired CRS
            'maxPixels': 1e13  # Change to an appropriate value
        }

        # Export GLCM texture properties to Google Drive with export options
        export_glcm_contrast_task = ee.batch.Export.image.toDrive(
            image=glcm_mean_contrast,
            description='GLCM Contrast Export',
            folder=output_folder,
            fileNamePrefix=glcm_contrast_filename,
            **export_options  # Include export options
        )

        export_glcm_correlation_task = ee.batch.Export.image.toDrive(
            image=glcm_mean_correlation,
            description='GLCM Correlation Export',
            folder=output_folder,
            fileNamePrefix=glcm_correlation_filename,
            **export_options  # Include export options
        )

        export_glcm_dissimilarity_task = ee.batch.Export.image.toDrive(
            image=glcm_mean_dissimilarity,
            description='GLCM Dissimilarity Export',
            folder=output_folder,
            fileNamePrefix=glcm_dissimilarity_filename,
            **export_options  # Include export options
        )

        export_glcm_energy_task = ee.batch.Export.image.toDrive(
            image=glcm_mean_energy,
            description='GLCM Energy Export',
            folder=output_folder,
            fileNamePrefix=glcm_energy_filename,
            **export_options  # Include export options
        )

        export_glcm_entropy_task = ee.batch.Export.image.toDrive(
            image=glcm_mean_entropy,
            description='GLCM Entropy Export',
            folder=output_folder,
            fileNamePrefix=glcm_entropy_filename,
            **export_options  # Include export options
        )

        # Start export tasks
        export_glcm_contrast_task.start()
        export_glcm_correlation_task.start()
        export_glcm_dissimilarity_task.start()
        export_glcm_energy_task.start()
        export_glcm_entropy_task.start()

        print(f'Exporting GLCM texture properties for {year} to Google Drive...')

        # Wait for export tasks to complete
        while (export_glcm_contrast_task.active() or export_glcm_correlation_task.active()
               or export_glcm_dissimilarity_task.active() or export_glcm_energy_task.active()
               or export_glcm_entropy_task.active()):
            print('Exporting...')  # You can replace this with a more informative message
            time.sleep(30)  # Wait for 30 seconds before checking again

        print(f'Export completed for GLCM texture properties for {year}')
    elif not images_found:
        print(f"No images found for year {year}. Moving to the next year.")


{'type': 'Image', 'bands': [{'id': 'B4_asm', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B4_contrast', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B4_corr', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B4_var', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B4_idm', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B4_savg', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B4_svar', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B4_sent', 'data_ty

{'type': 'Image', 'bands': [{'id': 'B4_asm', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B4_contrast', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B4_corr', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B4_var', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B4_idm', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B4_savg', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B4_svar', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B4_sent', 'data_ty

Exporting...
Exporting...
Export completed for GLCM texture properties for 2009
{'type': 'Image', 'bands': [{'id': 'B4_asm', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B4_contrast', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B4_corr', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B4_var', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B4_idm', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B4_savg', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B4_svar', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'crs

Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Export completed for GLCM texture properties for 2012
{'type': 'Image', 'bands': [{'id': 'B4_asm', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B4_contrast', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B4_corr', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B4_var', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B4_idm', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B4_savg', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B4_svar', 'data_type': {'type': 'P